# DriftWatch Advanced Customization

Learn how to customize thresholds, select specific detectors, and fully customize visualizations.

In [ ]:
!pip install driftwatch[viz]

In [ ]:
import numpy as np
import pandas as pd

# Generate reference data (normal)
np.random.seed(42)
ref_data = pd.DataFrame(
    {
        "age": np.random.normal(30, 5, 1000),
        "income": np.random.exponential(50000, 1000),
        "category": np.random.choice(["A", "B", "C"], 1000, p=[0.5, 0.3, 0.2]),
    }
)
ref_data.to_parquet("reference.parquet")

# Generate current data (drifted)
curr_data = pd.DataFrame(
    {
        "age": np.random.normal(35, 5, 1000),  # Drifted mean
        "income": np.random.exponential(60000, 1000),  # Drifted scale
        "category": np.random.choice(
            ["A", "B", "C"], 1000, p=[0.3, 0.4, 0.3]
        ),  # Drifted probas
    }
)
curr_data.to_parquet("current.parquet")
print("Data generated: reference.parquet, current.parquet")

### 1. Custom Thresholds
Initialize the Monitor with a stricter global threshold.

In [ ]:
from driftwatch import Monitor

# Initialize with a global stricter threshold
monitor = Monitor(reference_data=ref_data, default_threshold=0.01)
report = monitor.check(curr_data)
print(f"Drift Detected: {report.has_drift()}")

### 2. Deep Dive with DriftExplainer
Analyze why drift is happening using statistical properties.

In [ ]:
from driftwatch.explain import DriftExplainer

explainer = DriftExplainer(ref_data)
explanations = explainer.explain(curr_data)

# Look at 'age' explanation
print(explanations["age"])

### 3. Visualizing with Customization
DriftWatch allows you to customize colors, labels, and plot styles.

In [ ]:
from driftwatch.explain import DriftVisualizer

# Initialize with custom base colors
viz = DriftVisualizer(
    reference_data=ref_data,
    production_data=curr_data,
    report=report,
    colors={"reference": "#2ecc71", "production": "#9b59b6"},  # Green vs Purple
)

# Plot with fully custom options
fig = viz.plot_feature(
    "age",
    title="Age Distribution Shift (Training vs Inference)",
    xlabel="Age (Years)",
    colors={
        "reference": "black",
        "production": "orange",
    },  # Override just for this plot
    hist_kwargs={"histtype": "stepfilled", "alpha": 0.5},
    stats_kwargs={"fontsize": 12, "facecolor": "white", "alpha": 1.0},
)

### 4. Plot All Features with Custom Style
Apply styles to the grid view as well.

In [ ]:
fig_all = viz.plot_all(
    cols=2,
    colors={"reference": "blue", "production": "red"},  # Red/Blue for grid
    hist_kwargs={"edgecolor": "black"},
)